# Part 3: Statistical association analysis

During this section of the tutorial you perform a Fisher's Exact association test to determine whether any TCR sequences are significantly <br>
enriched in the post-treatment repertoire compared to the pre-treatment repertoire.

In [1]:
# Import packages

import statsmodels.stats.multitest
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

from scipy.stats import fisher_exact


In [2]:
# Load the data

# Define the current working directory
cwd = '/your/working/directory'

# Read in the correct data files
P1_pre = pd.read_csv(f'{cwd}/Processed_data/P1_pre_data.tsv', sep='\t')
P1_post = pd.read_csv(f'{cwd}/Processed_data/P1_post_data.tsv', sep='\t')

P2_pre = pd.read_csv(f'{cwd}/Processed_data/P2_pre_data.tsv', sep='\t')
P2_post = pd.read_csv(f'{cwd}/Processed_data/P2_post_data.tsv', sep='\t')

P3_pre = pd.read_csv(f'{cwd}/Processed_data/P3_pre_data.tsv', sep='\t')
P3_post = pd.read_csv(f'{cwd}/Processed_data/P3_post_data.tsv', sep='\t')

In [3]:
# Merge pre- and post-treatment dataframes on overlapping TCR sequences
def get_dataframe(data_post, data_pre):
    data_pre.rename(columns={'Total_count': 'Count_pre'}, inplace=True)
    data_post.rename(columns={'Total_count': 'Count_post'}, inplace=True)
    dataframe = data_post.merge(data_pre, how='inner', on=['junction_aa', 'v_call', 'j_call', 'Full_CDR3'])
    return dataframe


def get_fisher_exact(data, CDR3):

    # Perform a Fisher's exact test per unique CDR3 sequence
    association = {"junction_aa":[],"p_value":[],"Odds_Ratio":[]}
    for i,j in zip(range(len(data)), CDR3):
        
        # Get clone count
        Pre = data['Count_pre'].iloc[i]
        Post = data['Count_post'].iloc[i]

        # Get total count for all other CDR3 sequences
        df = data.drop([i])
        rest_pre = df['Count_pre'].sum()
        rest_post = df['Count_post'].sum()

        # Do the Fisher exact test
        oddsr, p = fisher_exact([[Post, Pre],[rest_post, rest_pre]], alternative="greater")
        association["junction_aa"].append(j)
        association["p_value"].append(p)
        association['Odds_Ratio'].append(oddsr)

    return association


def get_bh_correction(Fisher_res):

    # Rank p-values and perform Benjamini-Hochberg multiple testing correction
    mtc = pd.DataFrame(Fisher_res).sort_values(by="p_value").set_index("junction_aa")
    BH_corr = statsmodels.stats.multitest.fdrcorrection(mtc['p_value'], alpha=0.05, method='indep', is_sorted=False)
    mtc['BH_p_values'] = BH_corr[1]

    # Select only significantly associated CDR3 sequences
    mtc = mtc[mtc.BH_p_values<0.05]

    return mtc


# Define full Fisher exact test and multiple testing correction
def Full_fisher_exact(data_post, data_pre):
    Data = get_dataframe(data_post, data_pre)
    CDR3_names = Data['Full_CDR3']
    Fisher_test = get_fisher_exact(Data, CDR3_names)
    Fisher_res = get_bh_correction(Fisher_test)

    return Fisher_res

In [4]:
# Execute Fisher association testing for all three patients.
Fisher_p1 = Full_fisher_exact(P1_post, P1_pre)
Fisher_p2 = Full_fisher_exact(P2_post, P2_pre)
Fisher_p3 = Full_fisher_exact(P3_post, P3_pre)

# Write to file for later use
Fisher_p1.to_csv(f'{cwd}/Processed_data/Fisher_p1.csv')
Fisher_p2.to_csv(f'{cwd}/Processed_data/Fisher_p2.csv')
Fisher_p3.to_csv(f'{cwd}/Processed_data/Fisher_p3.csv')

# See how many significantly increased TCR sequences are discovered per patient.
print('The number of significantly increased CDR3s in patient 1 is:', len(Fisher_p1))
print('The number of significantly increased CDR3s in patient 2 is:', len(Fisher_p2))
print('The number of significantly increased CDR3s in patient 3 is:', len(Fisher_p3))

The number of significantly increased CDR3s in patient 1 is: 48
The number of significantly increased CDR3s in patient 2 is: 751
The number of significantly increased CDR3s in patient 3 is: 113


In [5]:
# Fisher's Exact result example

Fisher_p1

,p_value,Odds_Ratio,BH_p_values
junction_aa,,,
CASSQDDWSVSSYNEQFF_TCRBV03_TCRBJ02,0.000000e+00,16029.528139,0.000000e+00
CASSQTAGGGANVLTF_TCRBV04_TCRBJ02,0.000000e+00,157.403033,0.000000e+00
CASSQVGAYGNEQFF_TCRBV04_TCRBJ02,8.762537e-77,219.902015,4.574044e-74
CATSEDGTGQETQYF_TCRBV24_TCRBJ02,1.904011e-50,15.449587,7.454202e-48
CASSQTGGKGTEAFF_TCRBV21_TCRBJ01,2.449059e-27,80.751096,7.670453e-25
CSAWTSGSASSYEQYF_TCRBV20_TCRBJ02,2.504279e-23,132.044923,6.536169e-21
CAISDTGSRTDTQYF_TCRBV10_TCRBJ02,4.168041e-21,5.004793,9.324503e-19
CSARLGFYEQYF_TCRBV20_TCRBJ02,8.350382e-21,6.125969,1.634587e-18
CASRAGTLSTDTQYF_TCRBV09_TCRBJ02,3.068745e-20,33.008766,5.339616e-18
